# Aula 3: Organização, Autenticação, Validação e MVC em APIs Python

Nesta aula abordaremos:

1. Organização do projeto
2. Padrão MVC aplicado a APIs
3. Autenticação de usuários
4. Validação de dados de entrada
5. Exemplo prático integrando tudo


## 1. Organização do Projeto

Uma boa organização facilita manutenção e escalabilidade. Estrutura recomendada:

```
my_api_project/
├── app/
│   ├── __init__.py        # Criação do app, configuração e registro de blueprints
│   ├── config.py          # Configurações de ambiente (dev, prod)
│   ├── models.py          # Definição de models (SQLAlchemy)
│   ├── schemas.py         # Definição de schemas para validação (Marshmallow)
│   ├── controllers/       # Lógica de rotas (controllers)
│   │   ├── auth.py
│   │   └── items.py
│   └── extensions.py      # Instância de extensões (JWT, DB)
├── migrations/            # Arquivos de migração de banco
├── requirements.txt       # Dependências
└── run.py                 # Ponto de entrada da aplicação
```


## 2. Padrão MVC (Model-View-Controller)

- **Model**: representa a camada de dados e regras de negócio (em `models.py`).
- **View**: camada de apresentação (em APIs, templates ou JSON responses em controllers).
- **Controller**: recebe requisições, invoca model e retorna response (blueprints em `controllers/`).

Essa separação melhora legibilidade e permite testes isolados.


## 3. Autenticação de Usuários

Usaremos **JWT** (JSON Web Tokens) com a extensão `Flask-JWT-Extended`.

```bash
pip install Flask-JWT-Extended
```


In [ ]:
# app/extensions.py
from flask_jwt_extended import JWTManager

jwt = JWTManager()


In [ ]:
# app/__init__.py
from flask import Flask
from .extensions import jwt
from .controllers.auth import auth_bp
from .controllers.items import items_bp

def create_app():
    app = Flask(__name__)
    app.config['JWT_SECRET_KEY'] = 'sua_chave_secreta_aqui'
    jwt.init_app(app)
    app.register_blueprint(auth_bp)
    app.register_blueprint(items_bp)
    return app


### 3.1. Rotas de Autenticação


In [ ]:
# app/controllers/auth.py
from flask import Blueprint, request, jsonify
from werkzeug.security import check_password_hash, generate_password_hash
from flask_jwt_extended import create_access_token

auth_bp = Blueprint('auth', __name__, url_prefix='/auth')

# Simulação de usuário em memória
users = {
    'murilo': {'password_hash': generate_password_hash('senha123')}
}

@auth_bp.route('/login', methods=['POST'])
def login():
    data = request.get_json()
    username = data.get('username')
    password = data.get('password')
    user = users.get(username)

    if not user or not check_password_hash(user['password_hash'], password):
        return jsonify({'msg': 'Usuário ou senha inválidos'}), 401

    access_token = create_access_token(identity=username)
    return jsonify({'access_token': access_token}), 200


### 3.2. Protegendo Rotas com @jwt_required


In [ ]:
# app/controllers/items.py
from flask import Blueprint, jsonify, request
from flask_jwt_extended import jwt_required, get_jwt_identity

items_bp = Blueprint('items', __name__, url_prefix='/items')

@items_bp.route('', methods=['GET'])
@jwt_required()
def list_items():
    current_user = get_jwt_identity()
    # Retorna itens associados ao usuário
    return jsonify({'user': current_user, 'items': ['Apple', 'Banana']}), 200


## 4. Validação de Dados de Entrada

Usaremos **Marshmallow** para definir schemas e validar JSON.

```bash
pip install marshmallow
```


In [ ]:
# app/schemas.py
from marshmallow import Schema, fields, validate, ValidationError

class ItemSchema(Schema):
    id = fields.Int(dump_only=True)
    name = fields.Str(required=True, validate=validate.Length(min=1))
    price = fields.Float(required=True)

item_schema = ItemSchema()
items_schema = ItemSchema(many=True)


### 4.1. Validação no Controller


In [ ]:
# Em app/controllers/items.py, adicione:
from flask import request, jsonify
from .schemas import item_schema, items_schema
from marshmallow import ValidationError

@items_bp.route('', methods=['POST'])
@jwt_required()
def create_item():
    json_data = request.get_json()
    try:
        data = item_schema.load(json_data)
    except ValidationError as err:
        return jsonify(err.messages), 400
    # Cria item com `data` validado...
    item = {'id': 3, **data}
    return jsonify(item), 201


## 5. Exemplo Prático Integrado

**run.py**:
```python
from app import create_app

app = create_app()

if __name__ == '__main__':
    app.run(debug=True)
```

Fluxo:
1. **POST /auth/login** com `{ "username": "murilo", "password": "senha123" }` → retorna token.
2. **POST /items** com header `Authorization: Bearer <token>` e JSON `{ "name": "Laranja", "price": 0.4 }` → valida e cria.
3. **GET /items** com token → lista itens do usuário.

Essa estrutura mostra MVC, autenticação JWT e validação de inputs.
